In [20]:
!pip install flask
import flask
from flask import Flask, render_template,request


Note: you may need to restart the kernel to use updated packages.


In [33]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import joblib
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from scipy.sparse import hstack

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.models import Model,load_model

from keras.layers import Input, Dense, BatchNormalization, Activation
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Input
from sklearn.metrics import mean_squared_log_error

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/agereddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/agereddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
app = Flask(__name__,template_folder="templates")

import pickle
model = load_model('baseline_mlp_model_epochs:001-val_loss:0.183.hdf5')

In [35]:
def prediction(input_, model):
    """this function predicts the price basing on the trained model"""
    batchsize = 256
    preds = model.predict(input_.tocsr(), batch_size=batchsize)
    preds = np.exp(preds)+1
    return preds

In [36]:
import math
def handle_data(data):
    def transform_category_name(category_name):
        try:
            main, sub1, sub2= category_name.split('/')
            return main, sub1, sub2
        except:
            return ('missing', 'missing', 'missing')
        
    def branded(brand_name):
        is_branded=[]
        for i in data['brand_name']:
            if i=='missing': 
                is_branded.append(0) 
            else: 
                is_branded.append(1)
            return is_branded

    if type(data)==type(pd.DataFrame()):   #checking if it is a dataframe or not
        #data['category_name'].fillna(value='missing',inplace=True)
        
        #data['category_main'], data['category_sub1'], data['category_sub2'] = zip(*data['category_name'].apply(transform_category_name))
        data['brand_name'].fillna(value='missing',inplace=True)
        data['is_branded']=zip(*data['brand_name'].apply(branded))
        data['item_description'].fillna(value='No description yet',inplace=True)
        data['name']= data['name']
        data['shipping']=data['shipping']
        data['item_condition_id']=data['item_condition_id']
        
        if 'price' in data.columns:
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id','price']]
        else:
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id']]
        
    else:      #if the given data is a data point
       # if type(data['category_name'])==type(float()):  #checking for nan values in category_name
            #data['category_name'].fillna("missing",inplace=True)
           # data['category_main'], data['category_sub1'], data['category_sub2'] = zip(*data['category_name'].apply(transform_category_name))

        if type(data['brand_name'])==type(float()):     #checking for nan values in brand_name
            data['brand_name'].fillna("missing",inplace=True)
            data['is_branded']=zip(*data['brand_name'].apply(branded))

        if type(data['item_description'])==type(float()):  #checking for nan values in item_description
            data['item_description'].fillna("No description yet",inplace=True)
            #data['category_main'], data['category_sub1'], data['category_sub2'] =data['category_name'].split('/')
        if data['brand_name']=='missing': 
            data["is_branded"]=0
        else: 
            data["is_branded"]=1
        data['name']= data['name']
        data['shipping']=data['shipping']
        data['item_condition_id']=data['item_condition_id']

        if 'price' in dict(data).keys():  #if price exits we will return it
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id','price']]
        else:
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id']]
    return False

In [37]:
def price_suggestion(X):

    X=handle_data(X)          
    if 'price' in dict(X).keys():
        target=X['price']  #checking if price contains in it or not
    else:
        target='we predicted it'
    vectorizer_name=pickle.load(open("vectorizer_name.pkl","rb"))
    bow_name=vectorizer_name.transform(X['name'].values)  #name vectorization
    
    vectorizer_item=pickle.load(open("vectorizer_item.pkl", 'rb'))
    bow_item=vectorizer_item.transform(X['item_description'].values) 

    vectorizer_cat=pickle.load(open("vectorizer_cat.pkl","rb"))
    bow_cat=vectorizer_cat.transform(X['category_main'].values) 
    
    vectorizer_sub1=pickle.load(open("vectorizer_sub1.pkl","rb"))
    bow_sub1=vectorizer_sub1.transform(X['category_sub1'].values)
    #text vectorization
    
    vectorizer_sub2=pickle.load(open("vectorizer_sub2.pkl","rb"))
    bow_sub2=vectorizer_sub2.transform(X['category_sub2'].values)
    #text vectorization
    
    vectorizer_brand=pickle.load(open("vectorizer_brand.pkl","rb"))
    bow_brand=vectorizer_brand.transform(X['brand_name'].values)  #text vectorization

    is_brand_oe=pickle.load(open("is_branded_oe.pkl","rb"))
    is_brand=is_brand_oe.transform(X[['is_branded']])
    
    item_cond_oe=pickle.load(open("item_cond.pkl","rb"))
    item_cond=item_cond_oe.transform(X[['item_condition_id']])
    
    shipping_oe=pickle.load(open("shipping_oe.pkl","rb"))
    shipping=shipping_oe.transform(X[['shipping']])
    


    concat=hstack([bow_name,bow_item,bow_cat,bow_sub1,bow_sub2,bow_brand,is_brand,item_cond,shipping])   #concatinating all the features
    best_model = load_model('baseline_mlp_model_epochs:001-val_loss:0.183.hdf5')

    #preds = best_model.predict(X_te.tocsr())[:, 0]
    predicted_price=[prediction(concat,best_model)]  #storing all the prices predicted by the four models
    return predicted_price,target  #ensembling taking mean out of four results.

In [38]:
@app.route('/')
def index():
    return flask.render_template('index.html')
    #return {'x':1,'y':2}
#To use the predict button in our web-app
@app.route('/predict',methods=["GET","POST"])
def predict():
    #For rendering results on HTML GUI
    data={}

    if request.method == 'POST':
        #form=request.form
        data['name']=request.form.get("name")
        data['item_description']=request.form.get("description")
        data['category_main']=request.form.get("Category")
        data['category_sub1']=request.form.get("Sub-Category1")
        data['category_sub2']=request.form.get("Sub-Category2")
        data['brand_name']=request.form.get("Brand")
        data['is_branded']=request.form.get("Is-Branded")
        data['shipping']=request.form.get("Shipping")
        data['item_condition_id']=request.form.get("Item Condition")
    
    df = pd.DataFrame([data], columns= ['name', 'item_description','category_main','category_sub1','category_sub2','brand_name','is_branded','shipping','item_condition_id']) 
    #features=[name,description,Category,Sub_Category1,Sub_Category2,Brand,Is_Branded,Shipping,Item_Condition]
    #int_features = [float(x) for x in request.form.values()]
    #final_features = [np.array(int_features)]
    print(df)
    predicted,target=price_suggestion(df)  #some random train data point.

    output = round(predicted[0][0][0], 2) 
    return render_template('index.html', prediction_text='The price is :{}'.format(output))

In [ ]:
from werkzeug.wrappers import Request, Response

if __name__ == '__main__':
    #app.debug=True   
    app.run(port=8081)
    #app.run(host='localhost', port=8081)

    
       

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2021-07-22 21:01:31,599] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/Users/agereddy/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/agereddy/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/agereddy/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/agereddy/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/agereddy/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/agereddy/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
   

In [29]:
python app.py runserver -d


SyntaxError: invalid syntax (<ipython-input-29-d35c366f0e11>, line 1)